In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from yolov5_infer import model
import os

COCO_TRAIN_LABELS = { 'person': [], 'bicycle': [], 'car': [], 'motorcycle': [], 'airplane': [], 'bus': [], 'train': [], 'truck': [], 
               'boat': [], 'traffic light': [], 'fire hydrant': [], 'stop sign': [], 'parking meter': [], 'bench': [], 'bird': [], 
               'cat': [], 'dog': [], 'horse': [], 'sheep': [], 'cow': [], 'elephant': [], 'bear': [], 'zebra': [], 'giraffe': [], 
               'backpack': [], 'umbrella': [], 'handbag': [], 'tie': [], 'suitcase': [], 'frisbee': [], 'skis': [], 'snowboard': [], 
               'sports ball': [], 'kite': [], 'baseball bat': [], 'baseball glove': [], 'skateboard': [], 'surfboard': [], 'tennis racket': [], 
               'bottle': [], 'wine glass': [], 'cup': [], 'fork': [], 'knife': [], 'spoon': [], 'bowl': [], 'banana': [], 'apple': [], 
               'sandwich': [], 'orange': [], 'broccoli': [], 'carrot': [], 'hot dog': [], 'pizza': [], 'donut': [], 'cake': [], 'chair': [], 
               'couch': [], 'potted plant': [], 'bed': [], 'dining table': [], 'toilet': [], 'tv': [], 'laptop': [], 'mouse': [], 'remote': [], 
               'keyboard': [], 'cell phone': [], 'microwave': [], 'oven': [], 'toaster': [], 'sink': [], 'refrigerator': [], 'book': [], 
               'clock': [], 'vase': [], 'scissors': [], 'teddy bear': [], 'hair drier': [], 'toothbrush': [] }
COCO_CLASS_NAMES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
            'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
            'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
            'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
            'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
            'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
            'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
            'hair drier', 'toothbrush']

COCO_ROOT = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/train/"

import os
for i in range(0, len(COCO_CLASS_NAMES)):
    imgs_path_train = COCO_ROOT + COCO_CLASS_NAMES[i]
    for root, dirs, files in os.walk(imgs_path_train):
        for file in files:
            COCO_TRAIN_LABELS[COCO_CLASS_NAMES[i]].append(os.path.join(root, file).split("//")[-1])

# Utils
def get_ground_truth(img_path):
    if '//' in img_path:
        img_path = img_path.split("//")[-1]
    if '/' in img_path:
        img_path = img_path.split("/")[-1]
    if '\\' in img_path:
        img_path = img_path.split("\\")[-1]

    for i in range(0, len(COCO_CLASS_NAMES)):
        result_string = ''.join(COCO_TRAIN_LABELS[COCO_CLASS_NAMES[i]])
        if img_path in result_string:
            return i
    return None    

In [108]:
# Confusion matrix for training set

true_labels_train = []
predicted_labels_train = []

cvt_lables = {
    'person' : 0,
    'cat' : 1,
    'dog' : 2,
    'other' : 3
    }

TRAIN_ROOT = 'C:/Users/phiho/Projects/GAN_Yolov5/Dataset/train_dog_cat_person'

for root, dirs, files in os.walk(TRAIN_ROOT):
    if len(files) > 0:
        for file in files:
            true_label_index = get_ground_truth(file)
            if true_label_index not in [0, 15, 16]:
                true_labels_train.append(3)
            else:
                if true_label_index == 0:
                    true_labels_train.append(0)
                elif true_label_index == 15:
                    true_labels_train.append(1)
                elif true_label_index == 16:
                    true_labels_train.append(2)
            
            predict_label_index = model(os.path.join(root, file))
            if predict_label_index not in [0, 15, 16]:
                predicted_labels_train.append(3)
            else:
                if predict_label_index == 0:
                    predicted_labels_train.append(0)
                elif predict_label_index == 15:
                    predicted_labels_train.append(1)
                elif predict_label_index == 16:
                    predicted_labels_train.append(2)

labels = ['person', 'cat', 'dog', 'other']
true_labels = np.array(true_labels_train)
predicted_labels = np.array(predicted_labels_train)

num_labels = len(labels)
cm = np.zeros((num_labels, num_labels), dtype=float)

label_to_index = {label: index for index, label in enumerate(labels)}

for true, pred in zip(true_labels, predicted_labels):
    cm[true, pred] += 1

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

df_cm = pd.DataFrame(cm_normalized, index=labels, columns=labels)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df_cm, annot=True, fmt='.2f', cmap='Reds')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix Heatmap')
plt.savefig('C:/Users/phiho/Projects/GAN_Yolov5/analysis/ConfusionMatrix_Train.png')
plt.close()

In [109]:
# Confusion matrix for testing set

true_labels_test = []
predicted_labels_test = []

cvt_lables = {
    'person' : 0,
    'cat' : 1,
    'dog' : 2,
    'other' : 3
    }

TEST_ROOT = 'C:/Users/phiho/Projects/GAN_Yolov5/Dataset/test_dog_cat_person'

for root, dirs, files in os.walk(TEST_ROOT):
    if len(files) > 0:
        for file in files:
            true_label_index = get_ground_truth(file)
            if true_label_index not in [0, 15, 16]:
                print(3)
                true_labels_test.append(3)
            else:
                if true_label_index == 0:
                    true_labels_test.append(0)
                elif true_label_index == 15:
                    true_labels_test.append(1)
                elif true_label_index == 16:
                    true_labels_test.append(2)
            
            predict_label_index = model(os.path.join(root, file))
            if predict_label_index not in [0, 15, 16]:
                predicted_labels_test.append(3)
            else:
                if predict_label_index == 0:
                    predicted_labels_test.append(0)
                elif predict_label_index == 15:
                    predicted_labels_test.append(1)
                elif predict_label_index == 16:
                    predicted_labels_test.append(2)

labels = ['person', 'cat', 'dog', 'other']
true_labels = np.array(true_labels_test)
predicted_labels = np.array(predicted_labels_test)

num_labels = len(labels)
cm = np.zeros((num_labels, num_labels), dtype=float)

label_to_index = {label: index for index, label in enumerate(labels)}

for true, pred in zip(true_labels, predicted_labels):
    cm[true, pred] += 1

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

df_cm = pd.DataFrame(cm_normalized, index=labels, columns=labels)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df_cm, annot=True, fmt='.2f', cmap='Reds')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix Heatmap')
plt.savefig('C:/Users/phiho/Projects/GAN_Yolov5/analysis/ConfusionMatrix_Test.png')
plt.close()

3
3
3
3
3
3
3
3
3
3
3
3


In [3]:
IMG_PREDICT_LABELS_PATH = "C:/Users/phiho/Projects/GAN_Yolov5/analysis/COCO_TRAIN_LABELS.txt"

import json

def load_json_from_txt(path):
    with open(path, 'r') as file:
        data = file.read()
    return json.loads(data)

def get_predict_label(file):
    for key, array in IMG_PREDICT_LABELS.items():
        if file in array:
            return key
    print('Error!')
    return None

IMG_PREDICT_LABELS = load_json_from_txt(IMG_PREDICT_LABELS_PATH)


In [13]:
def create_combined_dict(train_labels_dict, predict_labels_dict):
    combined_dict = {}
    
    for label, train_paths in train_labels_dict.items():
        for train_path in train_paths:
            image_name = os.path.basename(train_path)
            if image_name not in combined_dict:
                combined_dict[image_name] = [None, None]
            combined_dict[image_name][0] = label

    for label, predict_paths in predict_labels_dict.items():
        for predict_path in predict_paths:
            image_name = os.path.basename(predict_path)
            if image_name not in combined_dict:
                combined_dict[image_name] = [None, None]
            combined_dict[image_name][1] = label

    return combined_dict

dict_img = create_combined_dict(COCO_TRAIN_LABELS, IMG_PREDICT_LABELS)

In [21]:
for k, v in dict_img.items():
    print(k, v)
    break

000000000049_3.jpg ['person', 'person']
